In [2]:
import pandas as pd
from tqdm.notebook import tqdm

solutions = pd.read_csv("train_solutions_with_pyright.csv")
tasks = pd.read_excel("data/for_teams/train/tasks.xlsx")
tests = pd.read_excel("data/for_teams/train/tests.xlsx")

In [3]:
for error, answer in list(
    zip(
        solutions[solutions["message"].notna()]["message"].to_list(),
        solutions[solutions["message"].notna()]["author_comment"].to_list(),
    )
):
    print(
        f"""question: {error}
responce: {answer}"""
    )
    print("\n")

question: Expected ":"
responce: Вы забыли поставить двоеточие после условия.


question: "color" is not defined
responce: Необходимо использовать одинаковые названия переменных. 


question: Expected indented block
responce: Вы некорректно поставили отступы перед функцией print().


question: "(" was not closed
responce: Вы забыли поставить закрывающую скобку функции print().


question: Expected expression
responce: Проверьте написание метода, который хотите применить к спискам. 


question: Expected ":"
responce: Обратите внимание, что после else не нужно прописывать никаких условий.


question: "true" is not defined
responce: Обратите внимание на неверный синтаксис булевых значений.


question: "logoproject" is not defined
responce: Обратите внимание на неверный синтаксис переменных.


question: Expected indented block
responce: Обратите внимание на ошибку в табуляции выражения break.


question: Expected ":"
responce: Вы забыли поставить двоеточие после условия в цикле while.


qu

In [4]:
df = pd.merge(solutions, tasks[["id", "description"]], left_on="task_id", right_on="id", how="left").drop(
    ["id_y", "id_x", "task_id"], axis=1
)
df

,Unnamed: 0,student_solution,author_comment,author_comment_embedding,message,line_number,line,description
0,0,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.2475823611021042 -1.3317935466766357 0.2535...,NaN,NaN,NaN,"Реализуйте программу, которая проверит, что цв..."
1,1,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.25970256328582764 -1.4550446271896362 0.333...,NaN,NaN,NaN,"Реализуйте программу, которая проверит, что цв..."
2,2,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.28293243050575256 -1.4774413108825684 0.243...,NaN,NaN,NaN,"Реализуйте программу, которая проверит, что цв..."
3,3,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Ошибка в открытых тестах. \n\nОбратите внимани...,-0.44602400064468384 -0.845210611820221 -0.109...,NaN,NaN,NaN,"Реализуйте программу, которая проверит, что цв..."
4,4,"logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4...",Вы забыли поставить двоеточие после условия.,-0.2322002649307251 -1.3137400150299072 -0.383...,"Expected "":""",7.0,else,"Реализуйте программу, которая проверит, что цв..."
...,...,...,...,...,...,...,...,...
342,342,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.2739786207675934 -1.7219487428665161 0.0072...,NaN,NaN,NaN,"Лолита изучает, какие студии дизайна чаще всег..."
343,343,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,"Лолита изучает, какие студии дизайна чаще всег..."
344,344,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,"Лолита изучает, какие студии дизайна чаще всег..."
345,345,"punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,-0.23899996280670166 -1.6277823448181152 -0.07...,NaN,NaN,NaN,"Лолита изучает, какие студии дизайна чаще всег..."


In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import fire
from llama_cpp import Llama


SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
print(SYSTEM_PROMPT)


def get_message_tokens(model, role, content):
    content = f"{role}\n{content}\n</s>"
    content = content.encode("utf-8")
    message_tokens = model.tokenize(content, special=True)
    return message_tokens


def get_system_tokens(model):
    system_message = {"role": "system", "content": SYSTEM_PROMPT}
    return get_message_tokens(model, **system_message)


model_path = "testestestest/model-q8_0.gguf"
# Initialize the model
model = Llama(
    model_path=model_path,
    n_ctx=8192,
    n_parts=1,
    verbose=True,
    n_gpu_layers=10,
    n_threads=16,
)


def get_response(model, prompt, n_ctx=8192, top_k=30, top_p=0.9, temperature=0.1, repeat_penalty=1.1):

    # Prepare the messages
    messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": prompt}]

    # Generate the response
    response = ""
    for part in model.create_chat_completion(
        messages,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repeat_penalty=repeat_penalty,
        stream=True,
    ):
        delta = part["choices"][0]["delta"]
        if "content" in delta:
            response += delta["content"]

    return response.strip()

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from testestestest/model-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u3

Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.


llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.8000 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0

In [6]:
def create_hint_prompt_task(description, student_solution):
    prompt = f"""Задание:
{description}

Попытка студента:
```python
{student_solution}
```
Объясни ошибку одним предложением.
"""
    return prompt


def create_hint_prompt_linter(student_solution, linter_message, error_line_number, error_line):
    prompt = f"""Код:
```python
{student_solution}
```

Строка с ошибкой:
{error_line}

Сообщение об ошибке:
{linter_message}

Объясни ошибку одним предложением"""
    return prompt

In [7]:
solutions[solutions["message"].notna()].iloc[[30, 37]]

,Unnamed: 0,id,task_id,student_solution,author_comment,author_comment_embedding,message,line_number,line
65,65,108,7,year = input()\n\nwith open('projects_file.csv...,"Вы использовали переменную, которая не была за...",-0.5023635029792786 -1.2875404357910156 0.0578...,"""file"" is not defined",3.0,for line in file:
84,84,224,14,result = 0\n\nwhile True:\n info = input()\...,Вы забыли поставить знак подчеркивания в кавычки.,-0.08178413659334183 -1.1025950908660889 -0.12...,"""_"" is not defined",7.0,if _ in info and info.islower():


In [8]:
responses = []
from tqdm.notebook import tqdm

for idx, row in tqdm(solutions[solutions["message"].notna()].iloc[[30, 37]].iterrows()):
    if pd.notna(row["message"]):
        prompt = create_hint_prompt_linter(row["student_solution"], row["message"], row["line_number"], row["line"])
    else:
        prompt = create_hint_prompt_task(row["description"], row["student_solution"])
    response = get_response(model, prompt)
    responses.append(response)

0it [00:00, ?it/s]

llama_perf_context_print:        load time =   11443.11 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   164 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    41 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  150725.39 ms /   205 tokens
Llama.generate: 52 prefix-match hit, remaining 91 prompt tokens to eval
llama_perf_context_print:        load time =   11443.11 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    91 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    47 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19353.01 ms /   138 tokens


In [9]:
responses

['Ошибка возникает из-за неправильного использования переменной `file`, которая должна быть `my_file` (созданную в контексте `with open()`), а не просто `file`.',
 'Ошибка состоит в том, что переменная "_" не была определена никуда и вызывает исключение "name \'_\' is not defined", так как интерпретатор Python не знает о какой именно переменной идет речь.']

In [19]:
from app.utils.metric import calculate_score
from app.utils.submit import embedding2string, get_sentence_embedding

In [39]:
calculate_score("Train_solution.csv", "data/for_teams/train/solutions.xlsx")

0.40778747200965887

In [9]:
solutions = pd.read_csv("test_solutions_with_pyright.csv")
tasks = pd.read_excel("data/for_teams/test/tasks.xlsx")
tests = pd.read_excel("data/for_teams/test/tests.xlsx")

In [10]:
df_test = pd.merge(solutions, tasks[["id", "description"]], left_on="task_id", right_on="id", how="left").drop(
    ["id_y", "id_x", "task_id"], axis=1
)
df_test

,Unnamed: 0,student_solution,author_comment,author_comment_embedding,message,line_number,line,description
0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,"Реализуйте программу, которая напечатает стоим..."
1,1,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"""("" was not closed",5.0,print(f'Реализация проекта будет стоить {money...,"Реализуйте программу, которая напечатает стоим..."
2,2,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,"""("" was not closed",5.0,print(fРеализация проекта будет стоить {money}...,"Реализуйте программу, которая напечатает стоим..."
3,3,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,"Реализуйте программу, которая напечатает стоим..."
4,4,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,"Реализуйте программу, которая напечатает стоим..."
...,...,...,...,...,...,...,...,...
320,320,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,"Напишите функцию, определяющую успешна студия ..."
321,321,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,"Напишите функцию, определяющую успешна студия ..."
322,322,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,"Напишите функцию, определяющую успешна студия ..."
323,323,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,"Напишите функцию, определяющую успешна студия ..."


In [ ]:
responses_test = []

for idx, row in tqdm(solutions[solutions["message"].notna()].iloc[[30, 37]].iterrows()):
    if pd.notna(row["message"]):
        prompt = create_hint_prompt_linter(row["student_solution"], row["message"], row["line_number"], row["line"])
    else:
        prompt = create_hint_prompt_task(row["description"], row["student_solution"])
    response = get_response(model, prompt)
    responses_test.append(response)

In [40]:
solutions = pd.read_excel("data/for_teams/test/solutions.xlsx")
solutions["author_comment"] = responses_test
solutions = solutions.rename({"id": "solution_id"}, axis=1)
solutions["author_comment_embedding"] = solutions["author_comment"].apply(
    lambda x: embedding2string(get_sentence_embedding(x))
)

In [44]:
solutions[["solution_id", "author_comment", "author_comment_embedding"]].to_csv("test_solution.csv", index=False)